In [ ]:
# Importeer benodigde modules
import os
import json
import pandas as pd
from pathlib import Path

# **Unsupported:** IMEC Extractie
Dit script is geschreven om de resultaten van het beeldherkenningsmodel van IMEC te verzamelen. Tijdens verschillende projecten is, naast het trainen van een model met Custom Vision of YOLOv8, ook een model getraind en ontwikkeld door onderzoekers van IMEC. Om de prestaties van dit model te testen, heeft IMEC validaties uitgevoerd en de resultaten in de vorm van een JSON-bestand opgestuurd. Dit script heeft als doel deze resultaten uit te lezen en op te slaan in een bewerkbaar Excel-formaat.

---
<blockquote style="border-left: 5px solid red; padding-left: 10px;">

## **Attentie!** 
<code> Update onderstaande cell voordat je de het script runt </code></blockquote>
 

In [ ]:
# Input map met de Json file met labels die je gedownload hebt van Labelbox.
JsonFile : list[str] = [r"pad/naar/jouw/Json/Resultaten.json", r"pad/naar/jouw/andere/Json/Resultaten.json"]
# Bijvoorbeeld: [r"pad/naar/jouw/Json/Resultaten.json", r"pad/naar/jouw/andere/Json/Resultaten.json"]

# Output bestand waar de nieuwe Excel worden opgeslagen.
OutputExcelLocation : str = r"pad/naar/jouw/output/bestand.xlsx"
# Bijvoorbeeld: r"pad/naar/jouw/output/bestand.xlsx"


# Het scheidingsteken dat wordt gebruikt in de naam tussen de variabelen. 
# ScheidingsTeken = "-" zou er uit zien als "0002-A52-flits-1_0_0"
ScheidingsTeken : str = "-" 
# Bijvoorbeeld: "-" of "_" of " " etc.

---
## **Code**

In [ ]:
OutputExcel = pd.DataFrame()
InputJson = pd.DataFrame()

# Leest de json file in en maakt er een pd tabel van 
for JsonFile in JsonFile:
    Tijdelijk = pd.DataFrame()

    with open(JsonFile) as a:
        Tijdelijk = json.load(a)
    
    Tijdelijk = pd.DataFrame.from_dict(Tijdelijk, orient="index")
    InputJson =  pd.concat([InputJson, Tijdelijk])

# Leest de bestandsnamen uit de json en bouwd hiermee het frame van de output excel
InputJson.insert(0, "SampleName", "*")
for Index, Row in InputJson.iterrows():
    IndexName = Path(Index)
    IndexName = IndexName.stem.rsplit("_", 1)[0]
    InputJson.loc[Index,"SampleName"]= IndexName

InputJson= InputJson.reset_index(drop=True)

# Extraheer de Resultaten uit de input json 
for Index, Row in InputJson.iterrows():
    DfRows = []
    for Num, Data in enumerate(Row):
        if isinstance(Data, dict):
            InputRow = pd.DataFrame([Data])
            DfRows.append(InputRow)
    
    # Voor de gevonden resultaten structureer de data in een gestandaardiseerde manier voor dit project
    if DfRows:
        DfConcat = pd.concat(DfRows, ignore_index=True)

        DfConcat.rename(columns={'object_tp': 'True Positives', 'object_fp': 'False Positives', 'object_fn': 'False Negatives', "class_name":"class"}, inplace=True)

        # bereken de aantallen originele annotaties
        DfConcat["Mens: Instances"] = DfConcat["True Positives"] + DfConcat["False Negatives"]
        DfConcat["Model: Instances"] = DfConcat["True Positives"] + DfConcat["False Positives"]

        # Reorder de kolommen
        DfConcat = DfConcat[["class", "Mens: Instances", "Model: Instances", "True Positives", "False Positives", "False Negatives"]]
        
        
        DfConcat.loc[len(DfConcat)] = [
            "Ongewogen gemiddelden",
            DfConcat["Mens: Instances"].sum(),
            DfConcat["Model: Instances"].sum(),
            DfConcat["True Positives"].sum(),
            DfConcat["False Positives"].sum(),
            DfConcat["False Negatives"].sum(),    
        ]
        
        DfConcat.insert(0, "SampleName", Row["SampleName"])

        # Splitst de naam op het opgegeven scheidingsteken en voegt ze individueel toe. zo kunnen resultaten per variabel bekeken worden
        for Num, Var in enumerate(DfConcat.iloc[0,0].split(ScheidingsTeken)):
            DfConcat.insert(Num + 1, f"Variabele {Num + 1}", Var)   

        # Voegt de dfConcat toe aan de output excel. 
        # checkt op het aantal kolommen zodat de tabel met de meeste kolommen word aangevuld met de tabel met minder.
        # Vooral van belang bij de eerste data toevoegen om geen kolommen te verliezen
        if len(OutputExcel.columns) >= len(DfConcat.columns):
            OutputExcel = pd.concat([OutputExcel, DfConcat], ignore_index=True)
        else:
            OutputExcel = pd.concat([DfConcat, OutputExcel], ignore_index=True)

# Bereken de gewenste metrics op basis van resultaten
# PAS OP! resultaten maar p of r 0 kunnen niet goed berekend worden en de resultaten van deze samples kunnen niet vertrouwd worden.
# Wanneer dit veel voorkomt kan er voor gekozen worden om resultaten van specifieke variabel combinaties samen te voegen.
OutputExcel["Precision"] = OutputExcel["True Positives"]/(OutputExcel["True Positives"]+OutputExcel["False Positives"])
OutputExcel["Recall"] = OutputExcel["True Positives"]/(OutputExcel["True Positives"]+OutputExcel["False Negatives"])

OutputExcel["F1-score"] = (2*(OutputExcel["Precision"]*OutputExcel["Recall"]))/(OutputExcel["Precision"]+OutputExcel["Recall"])

# Sla de output op
if OutputExcelLocation.endswith(".xlsx"):
    OutputExcel.to_excel(OutputExcelLocation, index=False)
else:
    OutputExcel.to_excel(os.path.join(OutputExcelLocation, "IMECresultaten.xlsx"), index=False)